<a href="https://colab.research.google.com/github/Aditi0102/ML-notebook/blob/main/AiModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
! pip install -q kaggle

In [4]:
%tensorflow_version 2.x

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from IPython.display import clear_output
import cv2
from tensorflow.keras.optimizers import RMSprop
from google.colab.patches import cv2_imshow
import math

In [6]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"aditiagarwal01","key":"4c56849565c39183e765f285aeafe7d3"}\n'}

In [8]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [9]:
! kaggle datasets download -d mohamedmustafa/real-life-violence-situations-dataset

100% 3.57G/3.58G [01:18<00:00, 52.4MB/s]
100% 3.58G/3.58G [01:18<00:00, 48.8MB/s]


In [10]:
!unzip \*.zip  && rm *.zip
clear_output()
print("Extracted")

Extracted


rm: cannot remove 'v': Is a directory


In [11]:
import os
train_folder = 'Directories/training/violent'
if not os.path.exists(train_folder):
    os.makedirs(train_folder)

In [12]:
# Training Data

count = 0
for i in range (1,200):
    videoFile = f"/content/real life violence situations/Real Life Violence Dataset/Violence/V_{i}.mp4"
    cap = cv2.VideoCapture(videoFile)   # capturing the video from the given path
    print(f"Processing Violence_{i}")
    while True:
        ret, frame = cap.read()
        if ret is False:
            break
        frame = cv2.resize(frame, (64,64))
        fname = '%s/%d.jpg' % (train_folder, count)
        cv2.imwrite(fname, frame)
        count += 1
    
    cap.release()
clear_output()
print("Done Training with Violence !!")


Done Training with Violence !!


In [13]:
import os
test_folder = 'Directories/validation/violent'
if not os.path.exists(test_folder):
    os.makedirs(test_folder)

In [14]:
# Testing Data
count = 0
for i in range (200,300):
    videoFile = f"/content/real life violence situations/Real Life Violence Dataset/Violence/V_{i}.mp4"
    cap = cv2.VideoCapture(videoFile)   # capturing the video from the given path
    print(f"Processing Violence_{i}")
    while True:
        ret, frame = cap.read()
        if ret is False:
            break
        frame = cv2.resize(frame, (64,64))
        fname = '%s/%d.jpg' % (test_folder, count)
        cv2.imwrite(fname, frame)
        count += 1
    
    cap.release()
clear_output()
print("Done Training with Violence !!")


Done Training with Violence !!


In [15]:
import os
nv_folder = 'Directories/training/non_violent'
if not os.path.exists(nv_folder):
    os.makedirs(nv_folder)

In [16]:
count = 0
for i in range (1,200):
    videoFile = f"/content/real life violence situations/Real Life Violence Dataset/NonViolence/NV_{i}.mp4"
    cap = cv2.VideoCapture(videoFile)   # capturing the video from the given path
    print(f"Reading File NonViolence_{i}")
    while True:
        ret, frame = cap.read()
        if ret is False:
            break

        frame = cv2.resize(frame, (64,64))
        fname = '%s/%d.jpg' % (nv_folder, count)
        cv2.imwrite(fname, frame)
        count += 1

        count += 1
    cap.release()
clear_output()
print("Done Training with Non-Violence !!")

Done Training with Non-Violence !!


In [17]:
import os
nv_test_folder = 'Directories/validation/non_violent'
if not os.path.exists(nv_test_folder):
    os.makedirs(nv_test_folder)

In [18]:
count = 0
for i in range (200,300):
    videoFile = f"/content/real life violence situations/Real Life Violence Dataset/NonViolence/NV_{i}.mp4"
    cap = cv2.VideoCapture(videoFile)   # capturing the video from the given path
    print(f"Reading File NonViolence_{i}")
    while True:
        ret, frame = cap.read()
        if ret is False:
            break

        frame = cv2.resize(frame, (64,64))
        fname = '%s/%d.jpg' % (nv_test_folder, count)
        cv2.imwrite(fname, frame)
        count += 1

        count += 1
    cap.release()
clear_output()
print("Done Training with Non-Violence !!")


Done Training with Non-Violence !!


In [19]:

base_dir = './Directories'
train_dir = os.path.join(base_dir,'training')
validation_dir = os.path.join(base_dir,'validation')

In [20]:
train_violent_dir = os.path.join(train_dir,'violent')
train_nonviolent_dir=os.path.join(train_dir,'non_violent')
validation_violent_dir=os.path.join(validation_dir,'violent')
validation_nonviolent_dir=os.path.join(validation_dir,'non_violent')


In [21]:
train_datagen= ImageDataGenerator(rescale=1./255,rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, horizontal_flip=True, fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale =1./255)

In [22]:
train_generator = train_datagen.flow_from_directory(train_dir, target_size = (150,150),batch_size=20, class_mode='binary')

Found 54226 images belonging to 2 classes.


In [23]:
validation_generator = test_datagen.flow_from_directory(validation_dir,target_size=(150,150),batch_size=20,class_mode='binary')


Found 27123 images belonging to 2 classes.


In [24]:
model= tf.keras.models.Sequential([
       tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)),
       tf.keras.layers.MaxPooling2D(2,2),
       tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
       tf.keras.layers.MaxPooling2D(2,2),
       tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
       tf.keras.layers.MaxPooling2D(2,2),
       tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
       tf.keras.layers.MaxPooling2D(2,2),
       tf.keras.layers.Dropout(0.5),
       tf.keras.layers.Flatten(),
       tf.keras.layers.Dense(512, activation='relu'),
       tf.keras.layers.Dense(1,activation ='sigmoid')

])

In [25]:
model.compile(loss='binary_crossentropy',optimizer =RMSprop(lr=1e-4),metrics=['accuracy'])



In [26]:
model1= model.fit(train_generator, 
                  steps_per_epoch=100,
                  epochs=100, 
                  validation_data=validation_generator,
                  validation_steps=50,verbose=2)

Epoch 1/100
100/100 - 17s - loss: 0.5679 - accuracy: 0.6905 - val_loss: 0.9256 - val_accuracy: 0.5320
Epoch 2/100
100/100 - 10s - loss: 0.3792 - accuracy: 0.8315 - val_loss: 0.5958 - val_accuracy: 0.6810
Epoch 3/100
100/100 - 10s - loss: 0.3579 - accuracy: 0.8320 - val_loss: 0.7768 - val_accuracy: 0.6420
Epoch 4/100
100/100 - 10s - loss: 0.3296 - accuracy: 0.8555 - val_loss: 0.6260 - val_accuracy: 0.6730
Epoch 5/100
100/100 - 10s - loss: 0.3082 - accuracy: 0.8585 - val_loss: 0.5646 - val_accuracy: 0.7310
Epoch 6/100
100/100 - 10s - loss: 0.2710 - accuracy: 0.8855 - val_loss: 0.4967 - val_accuracy: 0.7570
Epoch 7/100
100/100 - 10s - loss: 0.2585 - accuracy: 0.8960 - val_loss: 0.7290 - val_accuracy: 0.7130
Epoch 8/100
100/100 - 10s - loss: 0.2423 - accuracy: 0.9005 - val_loss: 0.6461 - val_accuracy: 0.7530
Epoch 9/100
100/100 - 10s - loss: 0.2538 - accuracy: 0.9008 - val_loss: 1.0669 - val_accuracy: 0.6130
Epoch 10/100
100/100 - 10s - loss: 0.2447 - accuracy: 0.9025 - val_loss: 0.4524 - 

In [28]:
import time
t = time.time()
export_path_keras = "./{}.h5".format(int(t))
print(export_path_keras)

model.save(export_path_keras)

./1610992372.h5
